In [ ]:
import numpy as np
from skimage.metrics import structural_similarity
import cv2
from imageio import imread
from random import random, choice, randint, sample
from PIL import Image
from PIL import ImageDraw
import os


class GeneticAlgorithm:

    def __init__(self, img_path, save_name="temp", top=5, max_group=200, features=100, epochs=1000):
        self.original_img, self.img_type, self.row, self.col = self.open_image(img_path)
        self.max_group = max_group
        self.top = top
        self.save_name = save_name
        self.groups = []
        self.features = features
        self.epochs = epochs
        self.group_dict = dict()

        if not os.path.exists(save_name):
            os.mkdir(save_name)

        self.initialize_groups()

    def open_image(self, img_path):
        img = imread(img_path)
        row, col = img.shape[0], img.shape[1]
        return img, img_path.split(".")[-1], row, col

    def initialize_groups(self):
        print("Initializing...")
        for _ in range(self.max_group):
            group = []
            for _ in range(self.features):
                polygon = [[choice(np.linspace(0, self.row, self.features)), choice(np.linspace(0, self.col, self.features))] for _ in range(3)]
                polygon.append("#" + ''.join(choice('0123456789ABCDEF') for _ in range(6)))

                group.append(polygon.copy())

            self.groups.append(group.copy())
        print("Initialization complete!")

    def to_image(self, group):
        array = np.ndarray((self.original_img.shape[0], self.original_img.shape[1], self.original_img.shape[2]), np.uint8)
        array[:, :, 0] = 255
        array[:, :, 1] = 255
        array[:, :, 2] = 255
        new_image = Image.fromarray(array)
        draw = ImageDraw.Draw(new_image)
        for polygon in group:
            draw.polygon((polygon[0][0], polygon[0][1], polygon[1][0], polygon[1][1], polygon[2][0], polygon[2][1]), polygon[3])

        return new_image

    def calculate_similarity(self, group) -> float:
        generated_image = self.to_image(group)
        ssim = structural_similarity(np.array(self.original_img), np.array(generated_image), multichannel=True)
        return ssim

    def save_generated_image(self, group, epoch):
        image = self.to_image(group)
        image.save(os.path.join(self.save_name, str(epoch) + "." + self.img_type))

    def crossover(self, parent1, parent2) -> list:
        offspring1 = []
        offspring2 = []

        min_locate = min(len(parent1), len(parent2))
        crossover_point = randint(1, min_locate - 1)

        offspring1 = [-1] * min_locate
        offspring2 = [-1] * min_locate

        offspring1[:crossover_point] = parent1[:crossover_point]
        offspring2[:crossover_point] = parent2[:crossover_point]

        for i in parent2:
            if i not in offspring1:
                for j in range(crossover_point, min_locate):
                    if offspring1[j] == -1:
                        offspring1[j] = i
                        break

        for i in parent1:
            if i not in offspring2:
                for j in range(crossover_point, min_locate):
                    if offspring2[j] == -1:
                        offspring2[j] = i
                        break

        return offspring1, offspring2

    def mutate(self, group):
        mutation_probability = 0.2
        mutation_step_size = 0.1

        for i in range(len(group)):
            for j in range(3):
                if np.random.rand() < mutation_probability:
                    group[i][j][0] += np.random.normal(0, random()*mutation_step_size * self.row)
                    group[i][j][1] += np.random.normal(0, random()*mutation_step_size * self.col)
                    group[i][j][0] = np.clip(group[i][j][0], 0, self.row)
                    group[i][j][1] = np.clip(group[i][j][1], 0, self.col)

            if np.random.rand() < mutation_probability:
                color = group[i][-1]
                r, g, b = int(color[1:3], 16), int(color[3:5], 16), int(color[5:7], 16)

                r = np.clip(r + int(np.random.normal(0, random()*mutation_step_size * 255)), 0, 255)
                g = np.clip(g + int(np.random.normal(0, random()*mutation_step_size * 255)), 0, 255)
                b = np.clip(b + int(np.random.normal(0, random()*mutation_step_size * 255)), 0, 255)

                new_color = "#{:02X}{:02X}{:02X}".format(r, g, b)
                group[i][-1] = new_color

        return group

    def generate_variations(self, group):
        mutated = self.mutate(group.copy())
        return [mutated]

    def breed(self, parent1, parent2):
        offspring1, offspring2 = self.crossover(parent1.copy(), parent2.copy())

        offspring = []
        offspring.extend(self.generate_variations(parent1.copy()))
        offspring.extend(self.generate_variations(parent2.copy()))

        return offspring

    def eliminate_less_fit(self, groups):
        self.group_dict.clear()
        for idx in range(len(groups)):
            self.group_dict[idx] = self.calculate_similarity(groups[idx])

        self.group_dict = {key: value for key, value in
                          sorted(self.group_dict.items(), key=lambda item: item[1], reverse=True)}

        fittest_groups = []
        for key in list(self.group_dict.keys())[:self.max_group]:
            fittest_groups.append(self.groups[key].copy())

        groups = fittest_groups.copy()
        return groups, list(self.group_dict.values())[0]

    def run(self):
        # Run the genetic algorithm to recreate the input image
        self.eliminate_less_fit(self.groups)
        for epoch in range(self.epochs):
            # Breeding process
            breed_n = randint(self.max_group // 2, self.max_group)
            fitness_values = np.abs(np.array(list(self.group_dict.values())))
            probabilities = fitness_values / np.sum(fitness_values)
            for _ in range(breed_n):
                father_idx = np.random.choice(list(self.group_dict.keys()), p=probabilities.ravel())
                mother_idx = np.random.choice(list(self.group_dict.keys()), p=probabilities.ravel())
                if father_idx < self.max_group and mother_idx < self.max_group:
                    self.groups.extend(self.breed(self.groups[int(father_idx)].copy(), self.groups[int(mother_idx)].copy()))

            # Eliminate less fit groups
            self.groups, accuracy = self.eliminate_less_fit(self.groups.copy())
            print("Epochs :", epoch+1, " Accuracy:", accuracy)
            last_group = self.groups[0]

            if epoch % 100 == 0:
                self.save_generated_image(self.groups[0], epoch)

            if accuracy >= 0.95:
                break
        self.save_generated_image(self.groups[0], "End")


    # Define input parameters
path = '/content/test.png'
savename = 'test result'

# Create a Genetic_Algorithm instance and run the algorithm
population_size = 500
GA = GeneticAlgorithm(path, savename, 5, population_size, 50, 100000000)
GA.run()

<ipython-input-11-8e7ce36ff90a>:29: DeprecationWarning: Starting with ImageIO v3 the behavior of this function will switch to that of iio.v3.imread. To keep the current behavior (and make this warning disappear) use `import imageio.v2 as imageio` or call `imageio.v2.imread` directly.
  img = imread(img_path)


Initializing...
Initialization complete!


<ipython-input-11-8e7ce36ff90a>:60: FutureWarning: `multichannel` is a deprecated argument name for `structural_similarity`. It will be removed in version 1.0. Please use `channel_axis` instead.
  ssim = structural_similarity(np.array(self.original_img), np.array(generated_image), multichannel=True)


Epochs : 1  Accuracy: 0.38534567229817257
Epochs : 2  Accuracy: 0.37669692909443403
Epochs : 3  Accuracy: 0.3751083265708019
Epochs : 4  Accuracy: 0.36991501265086907
Epochs : 5  Accuracy: 0.4012676831486067
Epochs : 6  Accuracy: 0.40553701948192966
Epochs : 7  Accuracy: 0.37183789482852914
Epochs : 8  Accuracy: 0.3834692975612738
Epochs : 9  Accuracy: 0.35110565669324134
Epochs : 10  Accuracy: 0.3485718268836905
Epochs : 11  Accuracy: 0.3455799042174288
Epochs : 12  Accuracy: 0.34197524865758616
Epochs : 13  Accuracy: 0.34826914604026016
Epochs : 14  Accuracy: 0.3294677108078881
Epochs : 15  Accuracy: 0.3338370320663072
Epochs : 16  Accuracy: 0.30630729725433664
Epochs : 17  Accuracy: 0.2976565035296342
Epochs : 18  Accuracy: 0.30970118079811404
Epochs : 19  Accuracy: 0.28029783759488275
Epochs : 20  Accuracy: 0.2750508742810433
Epochs : 21  Accuracy: 0.2966115913964003
Epochs : 22  Accuracy: 0.229614872907434
Epochs : 23  Accuracy: 0.2923729042546334
Epochs : 24  Accuracy: 0.26070337